In [1]:
import numpy as np
import pandas as pd
import simple_return as sr
import statistics as sc
import stdev
from scipy.stats import kurtosis, skew
import matplotlib.pyplot as plt
from scipy.stats import norm
import pickle

In [2]:
dt=5./(250*360)
delta=dt
sdt=np.sqrt(dt)
x=pd.ExcelFile('/home/thegodfather/Desktop/IISER/Codes&Data2/NIFTY Indices_5-Minute Data_For Anindya.xlsx')

In [3]:
z=x.sheet_names
lz=len(z)
y=[]
for r in z:
    y.append(x.parse(r))
N=[len(y[i]) for i in range(lz)]

In [4]:
def rev(x):
    zz=[]
    for i in reversed(x):
        zz.append(i)
    return(zz)

In [5]:
price=[rev(np.array(y[j].Close)) for j in range(lz)]
ret_five=[sr.s_ret(np.array(price[j],dtype=float)) for j in range(lz)]
oldret=[]
for i in range(lz):
    oldret.append(ret_five[i].copy())
rbar=[np.mean(ret_five[j]) for j in range(lz)]

In [6]:
def vola(ret_m):
    ret_m=np.array(ret_m)
    L=len(ret_m)
    std_m=np.array([stdev.sd(np.array(ret_m[i],dtype=float)) for i in range(L)])
    volatility_m= std_m
    return(volatility_m)

In [7]:
SD=vola(ret_five)
SDSquare=SD**2

ret=ret_five[0]
rBar=rbar[0]
SD2=SDSquare[0]
n=N[0]

In [8]:
def CalNewRet(ret,rBar,SD2,n):
    V=0
    lambdaHat=0
    p=2*10**(-4)
    sigmaHat=0
    for i in range(20):
        sigmaHat=np.sqrt((SD2-(lambdaHat*delta*V))/delta)
        const =norm.ppf((1-(p/2)), loc=0, scale=1)*sigmaHat*sdt
        card=0
        flag=[0]*(n-1)
        for j in range(0,n-1):
    #         print(j)
            if((ret[j]-rBar)>const):
                card+=1
                flag[j]=1
        lambdaHat=((card)/(n*delta))
        rsum=0
        for j in range(0,n-1):
            if(flag[j]==1):
                rsum+=((ret[j]-rBar)**2)
        V=(rsum)/(card-1)
    #     print("Iteration {}".format(i+1))
    #     print("sigmaHat={}".format(sigmaHat))
    #     print("lambdaHat={}".format(lambdaHat))
    #     print("V={}".format(V))
    #     print("Card={}".format(card))
    #     print("C={}".format(const))
    #     print("\n")
    c=const
    x=0
    for r in ret:
        exc=r-rBar
        if(abs(exc)>c):
            x+=1
            l=ret.index(r)
            ret[l]=rBar
#             for i in range(l,len(ret)):
#                 ret[i]=ret[i]-exc
#     print(x)

In [9]:
for i in range(lz):
    CalNewRet(ret_five[i],rbar[i],SDSquare[i],N[i])

In [10]:
print(np.mean(oldret[0]))

1.3436213637286906e-05


In [11]:
print(np.mean((ret_five[0])))

5.836108388004129e-06


In [12]:
with open('/home/thegodfather/Desktop/IISER/Codes&Data2/oldreturns.pkl', 'wb') as f:
    pickle.dump(oldret, f)

max1=0
for i in range(lz):
    if(max1<len(ret_five[i])):
        max1=len(ret_five[i])

for i in range(lz):
    if(len(ret_five[i])<max1):
        diff=max1-len(ret_five[i])
        for j in range(len(ret_five[i]),len(ret_five[i])+diff):
            ret_five[i].append(0.0)

writer=pd.ExcelWriter('/home/thegodfather/Desktop/IISER/Codes&Data2/NewReturns.xlsx',engine='xlsxwriter')
df=pd.DataFrame({'RO1':ret_five[0],'RO2':ret_five[1],'RO3':ret_five[2],'RO4':ret_five[3],'RO5':ret_five[4],'RO6':ret_five[5],'RO7':ret_five[6],'RO8':ret_five[7],'RO9':ret_five[8],'R10':ret_five[9],'R11':ret_five[10],'R12':ret_five[11],'R13':ret_five[12],'R14':ret_five[13],'R15':ret_five[14],'R16':ret_five[15],'R17':ret_five[16],'R18':ret_five[17]},index=range(1,11174))
df.to_excel(writer,sheet_name='sheet')
writer.save()

In [13]:
with open('/home/thegodfather/Desktop/IISER/Codes&Data2/returns.pkl', 'wb') as f:
    pickle.dump(ret_five, f)

In [14]:
with open('/home/thegodfather/Desktop/IISER/Codes&Data2/nn.pkl', 'wb') as f:
    pickle.dump(N, f)

In [15]:
with open('/home/thegodfather/Desktop/IISER/Codes&Data2/lz.pkl', 'wb') as f:
    pickle.dump(lz, f)